## Data Pre-Processing

In [178]:
import pandas as pd
import numpy as np
import math
import time

In [179]:
data = pd.read_csv("adult.csv", names=["Age", "Workclass", "fnlwgt", "Education", "Edu_Num", "MaritalStatus", "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "AnnualSalary"])

min_support = 0.5
min_support = min_support*len(data)
len(data)

32561

In [180]:
min_support

16280.5

In [181]:
data.head()

,Age,Workclass,fnlwgt,Education,Edu_Num,MaritalStatus,Occupation,Relationship,Race,Sex,CapitalGain,CapitalLoss,HoursPerWeek,NativeCountry,AnnualSalary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [182]:
data['HoursPerWeekGroup'] = pd.cut(data.HoursPerWeek, bins=[1, 20, 40, 60, 80, 100], labels=['VeryLow', 'Low', 'Average', 'High', 'VeryHigh'])
data['AgeGroup'] = pd.cut(data.Age, bins=[1, 25, 40, 60, 80], labels=['Youth', 'Adult', 'Elderly', 'OldAged'])

In [183]:
data.drop(['Edu_Num', 'CapitalGain', 'CapitalLoss', 'Age', 'HoursPerWeek', 'fnlwgt'], axis=1, inplace=True)
data.head()

,Workclass,Education,MaritalStatus,Occupation,Relationship,Race,Sex,NativeCountry,AnnualSalary,HoursPerWeekGroup,AgeGroup
0,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,United-States,<=50K,Low,Adult
1,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,<=50K,VeryLow,Elderly
2,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,United-States,<=50K,Low,Adult
3,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,United-States,<=50K,Low,Elderly
4,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,Cuba,<=50K,Low,Adult


In [184]:
data['List of Items'] = data.apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

In [185]:
data['List of Items']

0         State-gov,  Bachelors,  Never-married,  Adm-c...
1         Self-emp-not-inc,  Bachelors,  Married-civ-sp...
2         Private,  HS-grad,  Divorced,  Handlers-clean...
3         Private,  11th,  Married-civ-spouse,  Handler...
4         Private,  Bachelors,  Married-civ-spouse,  Pr...
                               ...                        
32556     Private,  Assoc-acdm,  Married-civ-spouse,  T...
32557     Private,  HS-grad,  Married-civ-spouse,  Mach...
32558     Private,  HS-grad,  Widowed,  Adm-clerical,  ...
32559     Private,  HS-grad,  Never-married,  Adm-cleri...
32560     Self-emp-inc,  HS-grad,  Married-civ-spouse, ...
Name: List of Items, Length: 32561, dtype: object

In [186]:
# function to find Ck
def find_unique_transactions(df):
    num = 0
    ListToStore = []
    while num<df.shape[0]:
        for i in df['List of Items'][num].split(','):
            for j in i:
                if i not in ListToStore:
                    ListToStore.append(i)
        num+=1
    return ListToStore      

In [187]:
# Frequency of each element
def find_frequency_transactions(df, Transaction_List):
    ListToStore_dictionary = { item : 0 for item in Transaction_List }

    # print(ListToStore_dictionary)

    length = len(Transaction_List)
    row = 0

    while row < df.shape[0]:
        for i in ListToStore_dictionary:
            if i in df['List of Items'][row]:
                ListToStore_dictionary[i] = ListToStore_dictionary[i] + 1
        row += 1
    return ListToStore_dictionary

In [188]:
# making my dictionary to dataframe
def dict_to_dataframe(dictionary):
    Col1 = [[x] for x in list(dictionary.keys())]
    Col2 = list(dictionary.values())

    # creating a data frame out of these columns
    data = pd.DataFrame({'Items': Col1, 'Count': Col2})
    # C1 step
    return data

In [189]:
# L1 Step --> comparing with min_sup value
# 'data' is my data frame
def prune(data):
    L1 = data[data['Count'] >= min_support].reset_index(drop=True)
    return L1

In [190]:
# creating new association rules
# L1 --> the new dataframe created
def join_step(L1):
    list_new = []

    for i in range(0,len(L1['Items'])):    
        for j in range(i+1,len(L1['Items'])):
            # print(L1['Items'][i])
            # print(L1['Items'][j])
            if len([x for x in L1['Items'][i][0].split(',') if x not in L1['Items'][j][0].split(',')]) == 1:
                list_new.append(list(set(L1['Items'][i][0].split(',') + L1['Items'][j][0].split(','))))
    return list_new

In [191]:
# finding how many times the subset occurs in the itemset
def counting_subset_frequency(df, list_new):
    
    count = 0
    row = 0
    new_dict = {}
    #while row < df.shape[0]:
    for i in list_new:
        count = 0
        for j in range(0, df.shape[0]):
            if set(i).issubset(set(df['List of Items'][j].split(','))):
                count += 1
                new_dict[','.join(i)] = count
    return new_dict

In [192]:
# making my dictionary to dataframe
def make_dict(new_dict):
    Col1 = [[x] for x in list(new_dict.keys())]
    Col2 = list(new_dict.values())

    # creating a data frame out of these columns
    data = pd.DataFrame({'Items': Col1, 'Count': Col2})
    # C1 step
    return data

In [193]:
# Splitting into separate strings
# L2 = data[data['Count'] >= min_support].reset_index(drop=True)
# L2
def split_to_individual_strings(L2):    
    lst = []
    
    for i in L2['Items']:
        lst.append(i[0].split(','))
    
    L2['Items'] = lst
    return L2['Items']

In [194]:
# making my dictionary to dataframe
def make_dict(new_dict):
    Col1 = [[x] for x in list(new_dict.keys())]
    Col2 = list(new_dict.values())

    # creating a data frame out of these columns
    data = pd.DataFrame({'Items': Col1, 'Count': Col2})
    # C1 step
    return data

## Apriori Algorithm

In [198]:

def apriori(data, min_support):
    
    
    # function to find Ck
    list1 = find_unique_transactions(data) #op - list
    
    # creating a dictionary from the list with frequencies
    dict1 = find_frequency_transactions(data, list1) # op - dict
    
    # Convert dict to dataframe
    C1 = dict_to_dataframe(dict1) # op - dataframe
    
    # Performing Pruning
    L1 = prune(C1) #op - dataframe
    
    # Join step - Forming new Association rules
    while(len(L1) != 0):
        
        print(C1)
        print(L1)
        
        new_join = join_step(L1)
        # print(new_join)

        # Calculating subset frequencies
        freq_of_subset = counting_subset_frequency(data, new_join) # op - dict

        # Convert dict to dataframe
        C1 = dict_to_dataframe(freq_of_subset) # op - df

        # Performing Pruning
        L1 = prune(C1) # op - df
        

In [200]:
start = time.time()
apriori(data, min_support)
end = time.time()
print(end - start)

                      Items  Count
0              [ State-gov]   1298
1             [  Bachelors]   5355
2         [  Never-married]  10683
3          [  Adm-clerical]   3770
4         [  Not-in-family]   8305
..                      ...    ...
107                [  Hong]     20
108             [  Ireland]     24
109         [ Never-worked]      7
110             [  Hungary]     13
111  [  Holand-Netherlands]      1

[112 rows x 2 columns]
               Items  Count
0          [  White]  27816
1           [  Male]  21790
2  [  United-States]  29170
3          [  <=50K]  24720
4             [ Low]  20052
5         [ Private]  22696
                         Items  Count
0             [  Male,  White]  19174
1    [  United-States,  White]  25621
2            [  <=50K,  White]  20699
3               [  White, Low]  16535
4           [ Private,  White]  19404
5     [  United-States,  Male]  19488
6             [  <=50K,  Male]  15128
7                [  Male, Low]  12573
8            [ Pri

# Apriori Improvement

In [209]:
data = pd.read_csv("adult.csv", names=["Age", "Workclass", "fnlwgt", "Education", "Edu_Num", "MaritalStatus", "Occupation", "Relationship", "Race", "Sex", "CapitalGain", "CapitalLoss", "HoursPerWeek", "NativeCountry", "AnnualSalary"])



In [210]:
# sampling 1000 rows
ran = np.random.choice(32561, 1000, replace=False)
data = data.iloc[ran].reset_index(drop = True)
data
min_support = 0.5
min_support = min_support*len(data)

In [211]:
min_support = 0.5
min_support = min_support*len(data)
data['HoursPerWeekGroup'] = pd.cut(data.HoursPerWeek, bins=[1, 20, 40, 60, 80, 100], labels=['VeryLow', 'Low', 'Average', 'High', 'VeryHigh'])
data['AgeGroup'] = pd.cut(data.Age, bins=[1, 25, 40, 60, 80], labels=['Youth', 'Adult', 'Elderly', 'OldAged'])
data.drop(['Edu_Num', 'CapitalGain', 'CapitalLoss', 'Age', 'HoursPerWeek', 'fnlwgt'], axis=1, inplace=True)
data.head()
data['List of Items'] = data.apply(lambda x: ', '.join(x[x.notnull()]), axis = 1)

In [212]:
start = time.time()
apriori(data, min_support)
end = time.time()
print(end - start)

                     Items  Count
0             [ State-gov]     44
1            [  Bachelors]    148
2   [  Married-civ-spouse]    469
3       [  Prof-specialty]    134
4                 [  Wife]     43
..                     ...    ...
93           [  Nicaragua]      2
94                [  Laos]      1
95        [  Armed-Forces]      1
96             [  England]      1
97            [  Columbia]      1

[98 rows x 2 columns]
               Items  Count
0          [  White]    839
1  [  United-States]    896
2             [ Low]    639
3         [ Private]    701
4           [  Male]    683
5          [  <=50K]    761
                         Items  Count
0    [  United-States,  White]    773
1               [  White, Low]    522
2           [ Private,  White]    581
3             [  Male,  White]    592
4            [  <=50K,  White]    625
5       [  United-States, Low]    564
6   [  United-States, Private]    624
7     [  United-States,  Male]    610
8    [  United-States,  <=50K] 

# FP Growth

In [96]:
data = pd.read_csv('sample_data.csv')

In [97]:
data

,Transactions,List of Items
0,T1,"I1,I2,I3"
1,T2,"I2,I3,I4"
2,T3,"I4,I5"
3,T4,"I1,I2,I4"
4,T5,"I1,I2,I3,I5"
5,T6,"I1,I2,I3,I4"


In [140]:
# class FPGrowth():
    
#     def __init__(self, min_sup=0.3):
#         self.min_sup = min_sup
#         self.root = None
#         # Prefixes of itemsets in the FP Growth Tree
#         self.prefixes = {}
#         self.frequent_itemsets = []


# function to find Ck
def find_unique_transactions(df):
    num = 0
    ListToStore = []
    
    while num<df.shape[0]:
        
        for i in df['List of Items'][num].split(','):
            for j in i:
                if i not in ListToStore:
                    ListToStore.append(i)
                    
        num+=1
    return ListToStore      


# Frequency of each element
def find_frequency_transactions(df, Transaction_List):
    ListToStore_dictionary = { item : 0 for item in Transaction_List }

    # print(ListToStore_dictionary)

    length = len(Transaction_List)
    row = 0

    while row < df.shape[0]:
        
        for i in ListToStore_dictionary:
            if i in df['List of Items'][row]:
                ListToStore_dictionary[i] = ListToStore_dictionary[i] + 1
                
        row += 1
    return ListToStore_dictionary


def sort_transactions(dictionary):
    '''sort the dictionary with respect to its values'''
    sorted_transactions = dict(sorted(dictionary.items(), key=lambda x:x[1], reverse=True))
    return sorted_transactions

def sorting_transactions_descending(sorted_transactions, df):
    Sort_List_By_Sorted_Transactions = []
    itemset = [ x.split(',') for x in data['List of Items'].tolist()]
    
    for i in itemset:
        Sort_List_By_Sorted_Transactions.append([ j for j in sorted_transactions if j in i])

    df['SortedList'] = Sort_List_By_Sorted_Transactions
    return df



def fpgrowth(data):
    lst = find_unique_transactions(data)
    dicti = find_frequency_transactions(data, lst)
    a = sort_transactions(dicti)
    print(a)
    b = sorting_transactions_descending(a, data)
    print(b)
    print(type(b))
    return b

In [141]:
b = fpgrowth(data)

{'I2': 5, 'I1': 4, 'I3': 4, 'I4': 4, 'I5': 2}
  Transactions List of Items        SortedList
0           T1      I1,I2,I3      [I2, I1, I3]
1           T2      I2,I3,I4      [I2, I3, I4]
2           T3         I4,I5          [I4, I5]
3           T4      I1,I2,I4      [I2, I1, I4]
4           T5   I1,I2,I3,I5  [I2, I1, I3, I5]
5           T6   I1,I2,I3,I4  [I2, I1, I3, I4]
<class 'pandas.core.frame.DataFrame'>


In [142]:
class TrieNode:
    """A node in the trie structure"""

    def __init__(self, item):
        # the character stored in this node
        self.item = item

        # whether this can be the end of a word
        self.is_end = False

        # a counter indicating how many times a word is inserted
        # (if this node's is_end is True)
        self.counter = 0

        # a dictionary of child nodes
        # keys are characters, values are nodes
        self.children = {}

In [143]:
class Trie(object):
    """The trie object"""

    def __init__(self):
        """
        The trie has at least the root node.
        The root node does not store any character
        """
        self.root = TrieNode("")
    
    def insert(self, itemset):
        """Insert a word into the trie"""
        node = self.root
        
        # Loop through each character in the word
        # Check if there is no child containing the character, create a new child for the current node
        for item in itemset:
            if item in node.children:
                node.children[item].counter+=1
                node = node.children[item]
            else:
                # If a character is not found,
                # create a new node in the trie
                new_node = TrieNode(item)
                node.children[item] = new_node
                node.children[item].counter= 1
                node = new_node
        
        # Mark the end of a word
        node.is_end = True

        # Increment the counter to indicate that we see this word once more
        node.counter += 1
        
    def dfs(self, node, prefix):
        """Depth-first traversal of the trie
        
        Args:
            - node: the node to start with
            - prefix: the current prefix, for tracing a
                word while traversing the trie
        """
        if node.is_end:
            self.output.append((prefix + node.char, node.counter))
        
        for child in node.children.values():
            self.dfs(child, prefix + node.char)


In [144]:
b

,Transactions,List of Items,SortedList
0,T1,"I1,I2,I3","[I2, I1, I3]"
1,T2,"I2,I3,I4","[I2, I3, I4]"
2,T3,"I4,I5","[I4, I5]"
3,T4,"I1,I2,I4","[I2, I1, I4]"
4,T5,"I1,I2,I3,I5","[I2, I1, I3, I5]"
5,T6,"I1,I2,I3,I4","[I2, I1, I3, I4]"


In [145]:
freq_pattern = Trie()

for i in range(len(b)):
    freq_pattern.insert(b['SortedList'][i])

In [147]:
freq_pattern.root.children

{'I2': <__main__.TrieNode at 0x22431943970>,
 'I4': <__main__.TrieNode at 0x2242f3cbc70>}

In [ ]:
def dfs(self, node, item):
        """Depth-first traversal of the trie
        
        Args:
            - node: the node to start with
            - prefix: the current prefix, for tracing a
                word while traversing the trie
        """
        if node.is_end:
            self.output.append((prefix + node.char, node.counter))
        
        for child in node.children.values():
            self.dfs(child, prefix + node.char)